In [1]:
import sqlite3
import pandas as pd
import numpy as np

import utils
from utils import TimedSQLiteConnection, get_query, get_query_one, get_query_dict, \
get_query_course_dict, get_query_df

conn = TimedSQLiteConnection('UMGC.db')
timedConnection = conn # so I can test function code directly

In [10]:
import re

def tokenize(expression):
    import re

    # Define the token pattern, accommodating course names
    token_pattern = r'\s*(\(|\)|\&|\||[A-Z]+\s+\d{3})\s*'
    tokens = re.findall(token_pattern, expression)
    return [token for token in tokens if token]

def parse(tokens):
    def _parse_expression(index):
        if tokens[index] == '(':
            sub_expressions = []
            index += 1
            while tokens[index] != ')':
                if tokens[index] in ('&', '|'):
                    operator = tokens[index]
                    index += 1
                else:
                    operand, index = _parse_expression(index)
                    sub_expressions.append(operand)
            index += 1
            if operator == '&':
                return ['and'] + sub_expressions, index
            elif operator == '|':
                return ['or'] + sub_expressions, index
        elif re.match(r'[A-Z]+\s+\d{3}', tokens[index]):
            return ['match', tokens[index]], index + 1
    
    expression_tree, _ = _parse_expression(0)
    return expression_tree

def parse_boolean_expression(expression):
    tokens = tokenize(expression)
    parsed_expression = parse(tokens)
    return parsed_expression

# Define your match function
def match(course):
    # Your logic to determine if the course matches
    # For this example, let's say it returns True if the course ends with '115'
    return course.endswith('115')

# Function to evaluate the parsed expression
def evaluate_expression_old(expression):
    if isinstance(expression, list):
        operator = expression[0]
        operands = expression[1:]
        
        if operator == 'and':
            return all(evaluate_expression(operand) for operand in operands)
        elif operator == 'or':
            return any(evaluate_expression(operand) for operand in operands)
    elif expression[0] == 'match':
        return match(expression[1])
    else:
        raise ValueError("Invalid expression")
    
# Function to evaluate the parsed expression and save intermediate results
def evaluate_expression_old(expression):
    intermediate_results = []  # List to store intermediate results
    
    def evaluate(expression):
        if isinstance(expression, list):
            operator = expression[0]
            operands = expression[1:]
            
            if operator == 'and':
                result = all(evaluate(operand) for operand in operands)
            elif operator == 'or':
                result = any(evaluate(operand) for operand in operands)
            
            intermediate_results.append((operator, result))  # Save intermediate result
            return result
        elif expression[0] == 'match':
            result = match(expression[1])
            intermediate_results.append(('match', result))  # Save intermediate result
            return result
        else:
            raise ValueError("Invalid expression")

    final_result = evaluate(expression)
    return final_result, intermediate_results

def evaluate_expression(expression):
    intermediate_results = []  # List to store intermediate results
    
    def evaluate(expression):
        nonlocal intermediate_results  # Declare intermediate_results as nonlocal
        
        if isinstance(expression, list):
            operator = expression[0]
            operands = expression[1:]
            
            if operator == 'and':
                result = all(evaluate(operand) for operand in operands)
            elif operator == 'or':
                result = any(evaluate(operand) for operand in operands)
            
            intermediate_results.append((operator, result))  # Save intermediate result
            return result
        elif expression[0] == 'match':
            result = match(expression[1])
            intermediate_results.append(('match', result))  # Save intermediate result
            return result
        else:
            raise ValueError("Invalid expression")

    final_result = evaluate(expression)
    return final_result, intermediate_results


In [ ]:
# Semester data
semester_data = []
seasons = ['SPRING', 'SUMMER', 'FALL']
for year in range(2024, 2041):
    for season in seasons:
        semester_data.append(f"{season} {year}")


In [ ]:
# Assign bins function
def assign_bins(max_credits, data):
    bin_credits = [0]
    course_bin = {}

    for item in data:
        if item['complete'] == 'true':
            course_bin[item['name']] = 0
            item['period'] = 0
            item['color'] = 'black'
            item['textcolor'] = 'white'
        elif item['complete'] == 'trans':
            course_bin[item['name']] = 0
            item['period'] = 0
            item['color'] = 'gray'
            item['textcolor'] = 'black'
        else:
            start_bin = 1
            if item['prerequisite']:
                prerequisites = item['prerequisite'].split(' & ')
                prerequisite_bins = [course_bin[prerequisite] for prerequisite in prerequisites]
                max_prerequisite_bin = max(prerequisite_bins)
                start_bin = max_prerequisite_bin + 1
            period = start_bin
            done = 0
            while done == 0:
                if period not in bin_credits:
                    bin_credits.append(item['credits'])
                    done = 1
                elif bin_credits[period] + item['credits'] > max_credits:
                    period += 1
                else:
                    bin_credits[period] += item['credits']
                    done = 1
            course_bin[item['name']] = period
            item['period'] = period

# Credit summary function
def credit_summary(data):
    summary = {}
    for item in data:
        if item['period'] not in summary:
            summary[item['period']] = {'totalCredits': 0, 'courseCount': 0}
        summary[item['period']]['totalCredits'] += item['credits']
        summary[item['period']]['courseCount'] += 1
    return summary


In [7]:
result

False

In [4]:
def match(course):
    print(f'This is the course: {course}\n')

In [12]:
!pip install graphlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for graphlib: filename=graphlib-0.9.5-py3-none-any.whl size=9634 sha256=a1f6ea9c0bd564c8a6261a4eed420f51b9d18bd04f27b93b2231ee8bf4ccd9cc
  Stored in directory: /Users/david/Library/Caches/pip/wheels/6e/97/aa/4d1b8530b8c0b43503611459bfaf81e7e90f8963bd761abcbd
Successfully built graphlib


In [13]:
from graphlib import TopologicalSorter

classes = {
    "A": [],
    "B": ["A"],
    "C": ["A"],
    "D": ["B", "C"],
    "E": ["A", "C"],
    "F": [],
    "G": [],
    "H": ["D"],
    "I": [],
    "J": ["H", "F"]
}

ts = TopologicalSorter(classes)

In [14]:
order = list(ts.static_order())

In [15]:
order

['A', 'F', 'G', 'I', 'B', 'C', 'D', 'E', 'H', 'J']